<a href="https://colab.research.google.com/github/ljrn8/NIDS-HPO/blob/main/colab%20notebookds/merge_datasets_may20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from pathlib import Path
import os, sys

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
root = Path('/content/drive') / 'MyDrive'  / 'datasets' / 'ids_datasets' / 'interm'
os.listdir(root)

['demo_botiot_clean.csv', 'full_cicids_cleaned.csv', 'full_unsw_cleaned.csv']

In [4]:
# bot = pd.read_csv(root / 'demo_botiot_clean.csv')
# print('loaded bot')
cicids = pd.read_csv(root / 'full_cicids_cleaned.csv')
print('loaded cicid')
unsw = pd.read_csv(root / 'full_unsw_cleaned.csv')
print('loaded unsw')

loaded cicid
loaded unsw


## Merge UNSW CICIDS

In [5]:
import pandas as pd

unsw_renamed = unsw.rename(columns={
    'dur': 'Duration',
    'sbytes': 'Fwd_Bytes',
    'dbytes': 'Bwd_Bytes',
    'Spkts': 'Fwd_Pkts',
    'Dpkts': 'Bwd_Pkts',
    'smeansz': 'Fwd_Pkt_Len_Mean',
    'dmeansz': 'Bwd_Pkt_Len_Mean',
    'swin': 'Fwd_InitWinBytes',
    'dwin': 'Bwd_InitWinBytes',
    'attack_cat': 'Target'
})

cicids_renamed = cicids.rename(columns={
    'Flow Duration': 'Duration',
    'Total Length of Fwd Packets': 'Fwd_Bytes',
    'Total Length of Bwd Packets': 'Bwd_Bytes',
    'Total Fwd Packets': 'Fwd_Pkts',
    'Total Backward Packets': 'Bwd_Pkts',
    'Fwd Packet Length Mean': 'Fwd_Pkt_Len_Mean',
    'Bwd Packet Length Mean': 'Bwd_Pkt_Len_Mean',
    'Init_Win_bytes_forward': 'Fwd_InitWinBytes',
    'Init_Win_bytes_backward': 'Bwd_InitWinBytes',
    'Label': 'Target'
})


common_cols = list(set(cicids_renamed.columns).intersection(set(unsw_renamed.columns)))
# common_cols.remove('Label') # now 'Target'

unsw_renamed['source'] = 'UNSW'
cicids_renamed['source'] = 'CICIDS'

In [6]:
for d in [cicids, unsw]: del d

# outer merge
outer_merge = pd.concat([unsw_renamed, cicids_renamed], axis=0, ignore_index=True, sort=False)

# readable col order
# cols = ['source', 'Target'] + [col for col in outer_merge.columns if col not in ['source', 'Target']]
# outer_merge = outer_merge[cols]

outer_merge.head()

,srcip,sport,dstip,dsport,proto,state,Duration,Fwd_Bytes,Bwd_Bytes,sttl,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label Binary
0,59.166.0.0,33661.0,149.171.126.9,1024.0,udp,CON,0.036133,528,304,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59.166.0.6,1464.0,149.171.126.7,53.0,udp,CON,0.001119,146,178,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59.166.0.5,3593.0,149.171.126.5,53.0,udp,CON,0.001209,132,164,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59.166.0.3,49664.0,149.171.126.0,53.0,udp,CON,0.001169,146,178,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59.166.0.0,32119.0,149.171.126.9,111.0,udp,CON,0.078339,568,312,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
outer_merge.to_csv(root / 'unsw_cicids_outer_merge_demos.csv')